In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import torch, wandb, os, pandas as pd 
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, resume_wandb_run, display_all, print_important_cfg_vars
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

import warnings
warnings.filterwarnings("ignore", message="Passing `max_length` to BeamSearchScorer is deprecated")  # we ignore the warning because it works anyway for diverse beam search 

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted Untitled.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted pp_eval_baselines.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted test_grammar_options.ipynb.
Converted test_pp_model.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any -- options when running with command line
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    for k,v in newargs.items(): 
        if v is not None: 
            if k in cfg.gen_params_train.keys():  cfg.gen_params_train[k] = v
            else:                   setattr(cfg, k, v)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer,vm_model,pp_tokenizer,pp_model,ref_pp_model,sts_model,nli_tokenizer,nli_model,cola_tokenizer,cola_model,cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=False)


travis_attack.data: INFO     Will load dataset simple with use_small_ds set to False
travis_attack.data: INFO     Will load dataset simple with use_small_ds set to False
travis_attack.data: INFO     Will load dataset simple with use_small_ds set to False


In [ ]:
cfg.wandb['mode'] = 'disabled'
trainer = Trainer(cfg, vm_tokenizer,vm_model,pp_tokenizer,pp_model,ref_pp_model,sts_model,nli_tokenizer,nli_model,cola_tokenizer,cola_model, optimizer,
         ds, initial_eval=True)
print_important_cfg_vars(cfg)
trainer.train()

{'pp_name': 'prithivida/parrot_paraphraser_on_T5',
 'dataset_name': 'simple',
 'sts_name': 'sentence-transformers/paraphrase-MiniLM-L12-v2',
 'nli_name': 'howey/electra-small-mnli',
 'cola_name': 'textattack/albert-base-v2-CoLA',
 'vm_name': 'textattack/distilbert-base-uncased-rotten-tomatoes',
 'seed': 421,
 'use_small_ds': False,
 'lr': 4e-05,
 'reward_fn': 'reward_fn_contradiction_and_letter_diff',
 'reward_clip_max': 4,
 'reward_vm_multiplier': 12,
 'sts_threshold': 0.8,
 'acceptability_threshold': 0.5,
 'contradiction_threshold': 0.2,
 'pp_letter_diff_threshold': 30,
 'reward_penalty_type': 'kl_div',
 'kl_coef': 0.25,
 'ref_logp_coef': 0.05,
 'max_pp_length': 48,
 'n_eval_seq': 48,
 'decode_method_train': 'sample',
 'decode_method_eval': 'sample',
 'gen_params_train': {'min_length': 2,
                      'max_length': 48,
                      'do_sample': True,
                      'temperature': 1.1,
                      'top_p': 0.95,
                      'length_penalty'

travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train
travis_attack.trainer: INFO     Launching initial eval run: train


travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid
travis_attack.trainer: INFO     Launching initial eval run: valid


travis_attack.trainer: INFO     Epoch: 0. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 0. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 0. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000


travis_attack.trainer: INFO     Launching initial eval run: test
travis_attack.trainer: INFO     Launching initial eval run: test
travis_attack.trainer: INFO     Launching initial eval run: test


...

travis_attack.trainer: INFO     Now on epoch 6 of 6
travis_attack.trainer: INFO     Now on epoch 6 of 6
travis_attack.trainer: INFO     Now on epoch 6 of 6


False False False False


travis_attack.trainer: INFO     Epoch: 6. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 6. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Epoch: 6. Min epochs before early stopping activated: 10
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000
travis_attack.trainer: INFO     Eval metric: 0.000 | Running median: 0.000


travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/dummy-36FPjQASsBexcLZydPyxTF/model_3.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/dummy-36FPjQASsBexcLZydPyxTF/model_3.pt
travis_attack.trainer: INFO     Evaluating test set with best model at path : ../model_checkpoints/travis_attack/dummy-36FPjQASsBexcLZydPyxTF/model_3.pt


True False False True


In [ ]:
df_expanded = df_expanded.drop_duplicates(subset=df_expanded.columns.difference(['pp_idx', 'sts_scores']))


 df_grp_stats = df_expanded[['idx'] + eval_metric_cols].groupby('idx').agg(agg_metrics)
df_grp_stats.columns = df_grp_stats.columns = ["-".join(a) for a in df_grp_stats.columns.to_flat_index()]
df_grp_stats = df_grp_stats.merge(df_expanded.groupby('idx').size().rename('n_pp').to_frame(), how='left', left_index=True, right_index=True)




df1 = df_expanded[0:10]
df1.drop_duplicates(subset=df1.columns.difference(['pp_idx', 'sts_scores']))

In [ ]:
%debug

> /home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/pandas/core/indexes/base.py(3623)get_loc()
   3621                 return self._engine.get_loc(casted_key)
   3622             except KeyError as err:
-> 3623                 raise KeyError(key) from err
   3624             except TypeError:
   3625                 # If we have a listlike key, _check_indexing_error will raise

ipdb> u 3 
> /data/tproth/travis_attack/travis_attack/trainer.py(115)_training_function()
    113         if self.initial_eval:
    114             logger.info("Launching initial eval run: train")
--> 115             self._eval_function(split='train')
    116             logger.info("Launching initial eval run: valid")
    117             self._eval_function(split='valid')

ipdb> d
> /data/tproth/travis_attack/travis_attack/trainer.py(733)_eval_function()
    731             for k in mean_only + mean_and_std:
    732                 name = k + "-mean"
--> 733                 wandb_eval_d[n

In [ ]:
#df = pd.read_csv(cfg.path_results + "run_results.csv")
#display_all(df)

In [ ]:
# df = pd.read_csv(f'{cfg.path_run}training_step.csv')
# #display_all(df.query('idx==1'))
# df.columns

In [ ]:
#trainer.run.finish()

In [ ]:
# # ## TO RESUME RUN
# cfg = Config()
# cfg.run_id = 'b9r88lmz'
# cfg.run_name = "fresh-cosmos-151"
# cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
# run = resume_wandb_run(cfg)


In [ ]:
#cfg.path_run

In [ ]:
df_d = get_training_dfs(cfg.path_run, postprocessed=False)
for k, df in df_d.items(): 
    df_d[k] = postprocess_df(df, filter_idx=None, num_proc=1)
    df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()
#run.finish()

travis_attack.insights: INFO     Dataframes have shapes ['training_step: (2445, 52)', 'train: (83456, 21)', 'valid: (10240, 21)', 'test: (1344, 21)']
travis_attack.insights: INFO     Adding text metrics for column orig


travis_attack.insights: INFO     Adding text metrics for column pp


travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


travis_attack.insights: INFO     Adding text metrics for column orig


travis_attack.insights: INFO     Adding text metrics for column pp


...

travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


travis_attack.insights: INFO     Adding text metrics for column orig


travis_attack.insights: INFO     Adding text metrics for column pp


travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


%_of_tokens_above_prob_1/vocab_size,████████████▁███████▁▁███▁██████████▁███
%_of_tokens_above_prob_1e-05,████████████▁███████▁▁███▁██████████▁███
%_of_tokens_above_prob_1e-06,████████████▁███████▁▁███▁██████████▁███
%_of_tokens_above_prob_1e-07,████████████▁███████▁▁███▁██████████▁███
%_of_tokens_above_prob_1e-08,████████████▁███████▁▁███▁██████████▁███
%_of_tokens_above_prob_1e-09,████████████▁███████▁▁███▁██████████▁███
acc_batch_n_examples,██▁██▁████▁██▁█▁██▁██▁█▁██▁████▁██▁█▁██▁
acc_num,▁▁▁▁████▁▁▁████▁▁▁████▁▁▁████▁▁▁████▁▁▁█
acceptability_scores-mean-test,█▁
acceptability_scores-mean-train,██▇▇▆▄▄▅▄▄▃▃▂▁▃▁
acceptability_scores-mean-valid,█▃▆▆▅▂▁▃▅▅▃▃▄▄▁▁
